In [ ]:
!pip install --upgrade scikit-learn

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [2]:
df_1 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_2 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [3]:
### Q1 ###

df_1.info()
df_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
### Q2 ###

df_1["duration"] = df_1["tpep_dropoff_datetime"] - df_1["tpep_pickup_datetime"]
df_1["duration"] = df_1["duration"].dt.total_seconds().div(60).astype(int)
np.std(df_1["duration"])

42.594492213415556

In [5]:
df_1["duration"].describe()

count    3.066766e+06
mean     1.518255e+01
std      4.259450e+01
min     -2.900000e+01
25%      7.000000e+00
50%      1.100000e+01
75%      1.800000e+01
max      1.002900e+04
Name: duration, dtype: float64

In [6]:
### Q3 ###

df_1_filtered = df_1[(df_1['duration'] >= 1) & (df_1['duration'] <= 60)]
df_1_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3011044 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [7]:
len(df_1_filtered)/len(df_1)

0.9818303711466737

In [8]:
### Q4 ###

df_1_filtered = df_1_filtered[["PULocationID", "DOLocationID", "duration"]]
df_1_filtered[["PULocationID", "DOLocationID"]] = df_1_filtered[["PULocationID", "DOLocationID"]].astype(str)
train_dicts = df_1_filtered[["PULocationID", "DOLocationID"]].to_dict(orient='records')

In [9]:
vectorizer = DictVectorizer()
vectorizer.fit(train_dicts)

DictVectorizer()

In [10]:
feature_mat = vectorizer.transform(train_dicts)

In [11]:
feature_mat.shape

(3011044, 515)

In [12]:
### Q5 ###

target = 'duration'
y_train = df_1_filtered[target].values

lr = LinearRegression()
lr.fit(feature_mat, y_train)

y_pred = lr.predict(feature_mat)

root_mean_squared_error(y_train, y_pred)

7.688651659305238

In [13]:
### Q6 ###

def process_dataframe(df):

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds().div(60).astype(int)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    features = ['PULocationID', 'DOLocationID']
    df[features] = df[features].astype(str)
    
    return df

In [14]:
df_val = process_dataframe(df_2)

/tmp/ipykernel_18609/891778666.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[features] = df[features].astype(str)


In [15]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2858178 entries, 0 to 2913954
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [16]:
val_dicts = df_val[["PULocationID", "DOLocationID"]].to_dict(orient='records')
X_val = vectorizer.transform(val_dicts)

In [17]:
y_pred = lr.predict(X_val)

target = 'duration'
y_val = df_val[target].values
root_mean_squared_error(y_val, y_pred)

7.868985034138532